In [1]:
!pip install mtcnn

In [2]:
import numpy as np
from glob import glob
import os
import cv2
import time
from mtcnn import MTCNN
import shutil

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
# img_folder_path = "drive/My Drive/Tibame_AIoT_Project/Datasets"
img_folder_path = "/content/drive/My Drive/Tibame_AIoT_Project/crowdhuman/yolos"
# src_path = 'cleandataset'
des_path = '/content/drive/My Drive/AIoT_Project/Datasets/crowdhuman_faces_yololabel'
# des_path = '/content/drive/My Drive/AIoT_Project/Datasets/cleandataset_yolo4mtcnn'

In [5]:
def convert_bndbox2yololabel(img,x,y,w,h):
    img_h = img.shape[0]
    img_w = img.shape[1]    

    x_center = (x + w/2.)/img_w
    y_center = (y + h/2.)/img_h
    w_yolo = w/img_w
    h_yolo = h/img_h

    return (x_center, y_center, w_yolo, h_yolo)

In [6]:
detector = MTCNN() 
print("detector:", type(detector))

detector: <class 'mtcnn.mtcnn.MTCNN'>


In [7]:
paths = glob(img_folder_path+ '/*.jpg')
img_folder_path, paths

('/content/drive/My Drive/Tibame_AIoT_Project/crowdhuman/yolos',
 ['/content/drive/My Drive/Tibame_AIoT_Project/crowdhuman/yolos/273278,d950f0001fd6e831.jpg',
  '/content/drive/My Drive/Tibame_AIoT_Project/crowdhuman/yolos/282555,1110380000f8c8548.jpg',
  '/content/drive/My Drive/Tibame_AIoT_Project/crowdhuman/yolos/273275,507930007830462b.jpg',
  '/content/drive/My Drive/Tibame_AIoT_Project/crowdhuman/yolos/273275,129cea000e4d1dc0a.jpg',
  '/content/drive/My Drive/Tibame_AIoT_Project/crowdhuman/yolos/283554,2cd1000094c469f0.jpg',
  '/content/drive/My Drive/Tibame_AIoT_Project/crowdhuman/yolos/273278,6bcdc00008c3e6f1.jpg',
  '/content/drive/My Drive/Tibame_AIoT_Project/crowdhuman/yolos/284193,19a95000944fe910.jpg',
  '/content/drive/My Drive/Tibame_AIoT_Project/crowdhuman/yolos/273275,e12af00023e2f786.jpg',
  '/content/drive/My Drive/Tibame_AIoT_Project/crowdhuman/yolos/284193,26aaf000a5f9e6d7.jpg',
  '/content/drive/My Drive/Tibame_AIoT_Project/crowdhuman/yolos/283554,3a7110003dc5b0bf

In [8]:
for i, path in enumerate(paths[2000:4000]):
    print(i, path)

    # 讀取圖片,切下臉的部分,並使用借來的模型的預處理方式來作預處理           
    img = cv2.imread(path)[:,:,::-1]
    # 取得 bounding boxes
    results = detector.detect_faces(img)
    if len(results) == 0:
        continue

    # 檔名
    fn = path.split('.')[0].split('/')[-1]	# for linux
    #fn = path.split('.')[0].split('\\')[-1]	# for windows
    print(fn)

    # 在 des_path 建立圖片對應的 txt file
    with open(os.path.join(des_path, fn +'.txt'), "w", encoding="utf-8") as f:
        pass

    # 取得每一張臉的 bounding box, and then convert to yolo's label 並寫入txt檔
    with open(os.path.join(des_path, fn +'.txt'), "a", encoding="utf-8") as f:
        for j, face in enumerate(results):
            x1, y1, width, height = results[j]['box']           
            yolo_label = convert_bndbox2yololabel(img, x1, y1, width, height)

            f.write("0 {:.06f} {:.06f} {:.06f} {:.06f}\n".format(yolo_label[0], yolo_label[1], yolo_label[2], yolo_label[3]))

    # 複製原始圖片到 des_path 
    shutil.copyfile( path, os.path.join(des_path, fn +'.jpg') )


/content/drive/My Drive/Tibame_AIoT_Project/crowdhuman/yolos/284193,216100018dfbf5f.jpg
284193,216100018dfbf5f
/content/drive/My Drive/Tibame_AIoT_Project/crowdhuman/yolos/273278,18b17000a1d75e94.jpg
273278,18b17000a1d75e94
/content/drive/My Drive/Tibame_AIoT_Project/crowdhuman/yolos/273275,3a28e00042da3a8c.jpg
273275,3a28e00042da3a8c
/content/drive/My Drive/Tibame_AIoT_Project/crowdhuman/yolos/273275,510dc000d0467239.jpg
273275,510dc000d0467239
/content/drive/My Drive/Tibame_AIoT_Project/crowdhuman/yolos/273278,f2726000dc9c49fa.jpg
273278,f2726000dc9c49fa
/content/drive/My Drive/Tibame_AIoT_Project/crowdhuman/yolos/283647,b240000aba99c11.jpg
283647,b240000aba99c11
/content/drive/My Drive/Tibame_AIoT_Project/crowdhuman/yolos/282555,266690007b254fbe.jpg
282555,266690007b254fbe
/content/drive/My Drive/Tibame_AIoT_Project/crowdhuman/yolos/273278,a76150005fb7decc.jpg
273278,a76150005fb7decc
/content/drive/My Drive/Tibame_AIoT_Project/crowdhuman/yolos/273275,d0140000325a41d7.jpg
273275,d014

In [9]:
new_jpgs = glob(des_path+'/*.jpg')
new_txts = glob(des_path+'/*.txt')
len(new_jpgs), len(new_txts)

(3279, 3279)